<a href="https://colab.research.google.com/github/MaguireMaName/Coursera_Capstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone: Battle of the Neighbourhoods

##Introduction/Business Problem

This project will bring together neighboured information on venues (from Foursquare) and neighbourhood crime statistics to find complementary neighbourhood concerning local venues, but to compare and contrast alike neighbourhoods with crime statistics.

This information will inform a business who wishes to set up shop in a reasonably safe neighbourhood or enable an individual to identify a venue to visit in a reasonably safe neighbourhood.

## Data description

The Foursquare data will be used to identify similar neighbourhoods by the frequency of venues by type. This alikeness will allow businesses to identify those neighbourhoods with relatively high levels of competition. For example, if I wanted to set up a coffee shop, I might want to avoid a neighbourhood saturated with coffee shops!

Furthermore, the value of the Foursquare dataset is to be extended by the inclusion of local crime dataset alongside resident population dataset. The integration of these datasets will enable business or individuals to identify further those venues and neighbourhoods that intersect with unreasonably high incidences of crime on a per capita basis.

This information will enable business or individuals to identify on a map those neighbourhoods based on the type of venues and the incidence of crime per capita.  Allowing businesses to target and assess the potential location of their operations by frequency of venues (i.e., competition) and low crime per capita.  Similarly, individuals can use the information to assess what venues to visit in what neighbourhood by type of venue and the incidence of crime in that neighbourhood.

In [0]:
#!pip install geocoder

In [0]:
# load dependancies

import pandas as pd
import numpy as np
import geocoder
import folium
from folium import plugins
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [7]:
# read in canberra neighbourhoods

cbr = pd.read_csv('Canberra suburbs.csv')
print(cbr.shape)
cbr.head()

(124, 4)


,Neighborhood,Postcode,Country,Region
0,ACTON,2601,Australia,Australian Capital Territory
1,AINSLIE,2602,Australia,Australian Capital Territory
2,AMAROO,2914,Australia,Australian Capital Territory
3,ARANDA,2614,Australia,Australian Capital Territory
4,BANKS,2906,Australia,Australian Capital Territory


In [0]:
Lat_list=[]
Lng_list=[]

for i in range(cbr.shape[0]):
    address='{}, Canberra, Australia'.format(cbr.at[i,'Neighborhood'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [9]:
for i in range(cbr.shape[0]): 
  
    cbr['Latitude'] = Lat_list
    cbr['Longitude'] = Lng_list
    
print(cbr.shape)
cbr.head()

(124, 6)


,Neighborhood,Postcode,Country,Region,Latitude,Longitude
0,ACTON,2601,Australia,Australian Capital Territory,-35.28562,149.11827
1,AINSLIE,2602,Australia,Australian Capital Territory,-35.26222,149.14655
2,AMAROO,2914,Australia,Australian Capital Territory,-35.16922,149.12637
3,ARANDA,2614,Australia,Australian Capital Territory,-35.25804,149.08293
4,BANKS,2906,Australia,Australian Capital Territory,-35.47004,149.09771


In [10]:
address = 'Canberra, Australian Capital Territory'

geolocator = Nominatim(user_agent="canberra_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Canberra using latitude and longitude values
map_cbr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(cbr['Latitude'], cbr['Longitude'], cbr['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(map_cbr)  
    
map_cbr

In [0]:
# define Foursquare Credentials and Version

client_id = 'KL5SVGOS40RKZBQK4G1VXYBKBICWCDQL2NMCASHFYER432SS' #  Foursquare ID'
client_secret = '1A5KPYJQIATH0SDZXPPZ5YK0SHLBYVEGPER5AAIIMDXLZ0AB' #  Foursquare Secret
version = '20180604'
limit = 100

In [0]:
# let's create a function to repeat the same process to all the neighborhoods in toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues

cbr_venues = getNearbyVenues(names=cbr['Neighborhood'],
                                   latitudes=cbr['Latitude'],
                                   longitudes=cbr['Longitude']
                                  )

ACTON
AINSLIE
AMAROO
ARANDA
BANKS
BARTON
BEARD
BELCONNEN
BLACK MOUNTAIN
BONNER
BONYTHON
BRADDON
BRUCE
CALWELL
CAMPBELL
CAPITAL HILL
CASEY
CHAPMAN
CHARNWOOD
CHIFLEY
CHISHOLM
CITY
CONDER
COOK
COOMBS
CRACE
CURTIN
DEAKIN
DENMAN PROSPECT
DICKSON
DOWNER
DUFFY
DUNLOP
DUNTROON
EVATT
FADDEN
FARRER
FISHER
FLOREY
FLYNN
FORDE
FORREST
FRANKLIN
FRASER
FYSHWICK
GARRAN
GILMORE
GIRALANG
GORDON
GOWRIE
GREENWAY
GRIFFITH
GUNGAHLIN
HACKETT
HALL
HARMAN
HARRISON
HAWKER
HIGGINS
HOLDER
HOLT
HUGHES
HUME
ISAACS
ISABELLA PLAINS
JACKA
KALEEN
KAMBAH
KENNY
KINGSTON
KINLYSIDE
LATHAM
LAWSON
LYNEHAM
LYONS
MACARTHUR
MACGREGOR
MACQUARIE
MAJURA
MAWSON
MCKELLAR
MELBA
MITCHELL
MOLONGLO
MONASH
MONCRIEFF
NARRABUNDAH
NGUNNAWAL
NICHOLLS
OAKS ESTATE
O'CONNOR
O'MALLEY
OXLEY
PAGE
PALMERSTON
PARKES
PEARCE
PHILLIP
PIALLIGO
RED HILL
REID
RICHARDSON
RIVETT
RUSSELL
SCULLIN
SPENCE
STIRLING
STROMLO
SYMONSTON
TAYLOR
THARWA
THEODORE
THROSBY
TORRENS
TURNER
URIARRA
WANNIASSA
WARAMANGA
WATSON
WEETANGERA
WESTON
WILLIAMSDALE
WRIGHT
YARRALUMLA


In [14]:
# check dimensions and data

print(cbr_venues.shape)
cbr_venues.head()

(667, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ACTON,-35.28562,149.11827,Monster Kitchen and Bar,-35.285122,149.122547,Hotel Bar
1,ACTON,-35.28562,149.11827,Palace Electric Cinema,-35.285014,149.123135,Movie Theater
2,ACTON,-35.28562,149.11827,National Film & Sound Archive,-35.283131,149.121143,Museum
3,ACTON,-35.28562,149.11827,Boffins,-35.283825,149.117795,Australian Restaurant
4,ACTON,-35.28562,149.11827,Ivy & The Fox,-35.288060,149.116116,Café


In [15]:
# the number of venues returned for each neighborhood

cbr_venues.groupby('Neighborhood').count()

print('There are {} unique venue categories.'.format(len(cbr_venues['Venue Category'].unique())))
print(cbr_venues.head())

There are 165 unique venue categories.
  Neighborhood  Neighborhood Latitude  ...  Venue Longitude         Venue Category
0        ACTON              -35.28562  ...       149.122547              Hotel Bar
1        ACTON              -35.28562  ...       149.123135          Movie Theater
2        ACTON              -35.28562  ...       149.121143                 Museum
3        ACTON              -35.28562  ...       149.117795  Australian Restaurant
4        ACTON              -35.28562  ...       149.116116                   Café

[5 rows x 7 columns]


In [16]:
# analyse each neighbourhood

# one hot encoding
cbr_onehot = pd.get_dummies(cbr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cbr_onehot['Neighborhood'] = cbr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cbr_onehot.columns[-1]] + list(cbr_onehot.columns[:-1])
cbr_onehot = cbr_onehot[fixed_columns]

cbr_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,...,Resort,Restaurant,River,Rugby Pitch,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ACTON,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
cbr_grouped = cbr_onehot.groupby('Neighborhood').mean().reset_index()
cbr_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,...,Resort,Restaurant,River,Rugby Pitch,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,ACTON,0.000000,0.00,0.125000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.000000,0.00,0.000,0.0,0.00,0.000000,0.000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000
1,AINSLIE,0.000000,0.00,0.125000,0.125000,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.125000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.000000,0.00,0.000,0.0,0.00,0.000000,0.125,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000
2,AMAROO,0.000000,1.00,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.000000,0.00,0.000,0.0,0.00,0.000000,0.000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000
3,ARANDA,0.000000,0.00,0.000000,0.000000,0.0,0.25,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.000000,0.00,0.000,0.0,0.00,0.000000,0.000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000
4,BANKS,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.00,0.00,0.0,0.000000,0.00,0.000,0.0,0.00,0.000000,0.000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000
5,BARTON,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.04,0.00,0.000000,0.04,0.0,0.000000,0.0,0.000000,0.040000,0.000000,0.00,0.200000,0.000000,0.04,0.000000,0.00,0.00,0.00,0.080000,0.00,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000

In [18]:
# top 3 frequencies

num_top_venues = 10

for hood in cbr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cbr_grouped[cbr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ACTON----
                   venue  freq
0                   Café  0.25
1  Australian Restaurant  0.12
2    Indie Movie Theater  0.12
3          Movie Theater  0.12
4              Hotel Bar  0.12
5                  Hotel  0.12
6                 Museum  0.12
7       Asian Restaurant  0.00
8                   Park  0.00
9              Nightclub  0.00


----AINSLIE----
                     venue  freq
0    Australian Restaurant  0.12
1                   Bakery  0.12
2                      Pub  0.12
3        Fish & Chips Shop  0.12
4                     Café  0.12
5            Grocery Store  0.12
6  Health & Beauty Service  0.12
7           Shopping Plaza  0.12
8         Asian Restaurant  0.00
9                     Park  0.00


----AMAROO----
                           venue  freq
0             Athletics & Sports   1.0
1                  Moving Target   0.0
2                         Museum   0.0
3                    Music Venue   0.0
4                      Nightclub   0.0
5            

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cbr_grouped['Neighborhood']

for ind in np.arange(cbr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ACTON,Café,Indie Movie Theater,Museum,Movie Theater,Hotel Bar,Australian Restaurant,Hotel,Food Court,Gaming Cafe,Furniture / Home Store
1,AINSLIE,Pub,Grocery Store,Australian Restaurant,Bakery,Café,Shopping Plaza,Fish & Chips Shop,Health & Beauty Service,Fruit & Vegetable Store,Frozen Yogurt Shop
2,AMAROO,Athletics & Sports,Yoga Studio,Food & Drink Shop,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,ARANDA,Recreation Center,Bar,Dance Studio,Café,Hot Dog Joint,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
4,BANKS,Grocery Store,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [21]:
# cluster neighbourhoods

# set number of clusters
kclusters = 4

cbr_grouped_clustering = cbr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([1, 1, 1, 1, 0], dtype=int32)

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

In [23]:
neighborhoods_venues_sorted.tail()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
105,WANNIASSA,1,Ice Cream Shop,Pool,Theater,Sandwich Place,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
106,WARAMANGA,0,Soccer Field,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
107,WATSON,1,Café,Filipino Restaurant,Fish & Chips Shop,Shopping Plaza,Grocery Store,French Restaurant,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
108,WESTON,1,Café,Fast Food Restaurant,Sandwich Place,Cricket Ground,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
109,WRIGHT,1,River,Yoga Studio,Flower Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [24]:
# merge dataset and check output

cbr_merged = pd.merge(cbr, neighborhoods_venues_sorted, on='Neighborhood')
cbr_merged.tail()

,Neighborhood,Postcode,Country,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
105,WANNIASSA,2903,Australia,Australian Capital Territory,-35.40288,149.09194,1,Ice Cream Shop,Pool,Theater,Sandwich Place,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
106,WARAMANGA,2611,Australia,Australian Capital Territory,-35.35331,149.06015,0,Soccer Field,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
107,WATSON,2602,Australia,Australian Capital Territory,-35.24132,149.15731,1,Café,Filipino Restaurant,Fish & Chips Shop,Shopping Plaza,Grocery Store,French Restaurant,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
108,WESTON,2611,Australia,Australian Capital Territory,-35.33660,149.05535,1,Café,Fast Food Restaurant,Sandwich Place,Cricket Ground,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
109,WRIGHT,2611,Australia,Australian Capital Territory,-35.32441,149.04089,1,River,Yoga Studio,Flower Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cbr_merged['Latitude'], cbr_merged['Longitude'], cbr_merged['Neighborhood'], cbr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
# cluster 0
cbr_merged.loc[cbr_merged['Cluster Labels'] == 0, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,2906,149.09771,0,Grocery Store,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
65,2615,149.03168,0,Soccer Field,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
68,2606,149.07544,0,Grocery Store,Café,Yoga Studio,Food Court,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
87,2913,149.11758,0,Grocery Store,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
94,2905,149.11124,0,Grocery Store,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
106,2611,149.06015,0,Soccer Field,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [27]:
# cluster 1
cbr_merged.loc[cbr_merged['Cluster Labels'] == 1, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2601,149.118270,1,Café,Indie Movie Theater,Museum,Movie Theater,Hotel Bar,Australian Restaurant,Hotel,Food Court,Gaming Cafe,Furniture / Home Store
1,2602,149.146550,1,Pub,Grocery Store,Australian Restaurant,Bakery,Café,Shopping Plaza,Fish & Chips Shop,Health & Beauty Service,Fruit & Vegetable Store,Frozen Yogurt Shop
2,2914,149.126370,1,Athletics & Sports,Yoga Studio,Food & Drink Shop,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,2614,149.082930,1,Recreation Center,Bar,Dance Studio,Café,Hot Dog Joint,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
5,2600,149.133540,1,Hotel,Café,Coffee Shop,Sports Bar,Gym,Hotel Bar,French Restaurant,Cantonese Restaurant,Steakhouse,Bus Station
6,2620,149.212170,1,Motel,Pool,Home Service,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
7,2617,149.063450,1,Coffee Shop,Café,Electronics Store,Music Venue,Burrito Place,Fast Food Restaurant,Middle Eastern Restaurant,Liquor Store,Malay Restaurant,Massage Studio
8,2601,149.097540,1,Mountain,Monument / Landmark,History Museum,Scenic Lookout,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
9,2914,149.143030,1,Brewery,Yoga Studio,Food Court,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
10,2905,149.080200,1,Wine Shop,IT Services,Cupcake Shop,Electronics Store,Event Space,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint


In [28]:
# cluster 2
cbr_merged.loc[cbr_merged['Cluster Labels'] == 2, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,2615,149.04430,2,Convenience Store,Garden Center,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop
42,2617,149.09891,2,Convenience Store,Garden Center,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop
102,2905,149.12418,2,Convenience Store,Garden Center,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


In [29]:
# cluster 3
cbr_merged.loc[cbr_merged['Cluster Labels'] == 3, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,2906,149.08503,3,Bus Station,Spa,Yoga Studio,Food Court,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
51,2914,149.16094,3,Park,Bus Station,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
79,2914,149.11349,3,Bus Station,Yoga Studio,Food Court,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
85,2903,149.08206,3,Bus Station,Yoga Studio,Food Court,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
96,2614,149.04042,3,Bus Station,Moving Target,Grocery Store,Garden Center,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
103,2607,149.08789,3,Bus Station,Café,Shop & Service,Gas Station,Garden,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint


In [30]:
# cluster 4
cbr_merged.loc[cbr_merged['Cluster Labels'] == 4, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [31]:
# cluster 5
cbr_merged.loc[cbr_merged['Cluster Labels'] == 5, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [32]:
# read in canberra neighbourhoods

cbr_crime = pd.read_csv('Canberra Suburb Crime Stats.csv')
print(cbr_crime)
cbr_crime.head()

       Neighbourhood  ...  Sum of 91d TINs Other
0              ACTON  ...                     81
1            AINSLIE  ...                     34
2             AMAROO  ...                     17
3             ARANDA  ...                    127
4              BANKS  ...                      4
5             BARTON  ...                     11
6              BEARD  ...                      1
7          BELCONNEN  ...                    283
8     BLACK MOUNTAIN  ...                      0
9             BONNER  ...                      8
10          BONYTHON  ...                     23
11           BRADDON  ...                     95
12             BRUCE  ...                    224
13           CALWELL  ...                     15
14          CAMPBELL  ...                     21
15      CAPITAL HILL  ...                      2
16             CASEY  ...                     29
17           CHAPMAN  ...                      5
18         CHARNWOOD  ...                     20
19           CHIFLEY

,Neighbourhood,Homicide,Assault_FV,Sum of 2b Assault -Non-FV,Sum of 3 Sexual Assault,Sum of 5a Robbery -armed,Sum of 4 Other offences against a person,Sum of 5b Robbery -�other,Sum of 6a Burglary dwellings,Sum of 6b Burglary shops,Sum of 6c Burglary other,Sum of 7 Motor vehicle�theft,Sum of 9 Other offences,Sum of 8 Property damage,Sum of 91a TINs Speeding,Sum of 91b TINs Mobile�Use,Sum of 91c TINs Seatbelts,Sum of 92 Theft (excluding�Motor Vehicles),Sum of 91d TINs Other
0,ACTON,4,4,18,1,1,1,0,1,2,6,12,122,50,68,20,3,81,81
1,AINSLIE,0,15,21,1,0,1,2,34,1,8,14,218,78,3,2,0,136,34
2,AMAROO,0,9,19,5,0,1,0,26,0,3,7,49,28,4,2,3,83,17
3,ARANDA,0,8,3,1,0,2,1,12,0,3,3,103,13,199,14,3,30,127
4,BANKS,0,5,2,3,0,1,0,16,1,0,8,59,33,1,0,0,35,4


In [0]:
Lat_list=[]
Lng_list=[]

for i in range(cbr_crime.shape[0]):
    address='{}, Canberra, Australia'.format(cbr_crime.at[i,'Neighbourhood'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [34]:
for i in range(cbr_crime.shape[0]): 
  
    cbr_crime['Latitude'] = Lat_list
    cbr_crime['Longitude'] = Lng_list
    
print(cbr_crime.shape)
cbr_crime.head()

(124, 21)


,Neighbourhood,Homicide,Assault_FV,Sum of 2b Assault -Non-FV,Sum of 3 Sexual Assault,Sum of 5a Robbery -armed,Sum of 4 Other offences against a person,Sum of 5b Robbery -�other,Sum of 6a Burglary dwellings,Sum of 6b Burglary shops,Sum of 6c Burglary other,Sum of 7 Motor vehicle�theft,Sum of 9 Other offences,Sum of 8 Property damage,Sum of 91a TINs Speeding,Sum of 91b TINs Mobile�Use,Sum of 91c TINs Seatbelts,Sum of 92 Theft (excluding�Motor Vehicles),Sum of 91d TINs Other,Latitude,Longitude
0,ACTON,4,4,18,1,1,1,0,1,2,6,12,122,50,68,20,3,81,81,-35.28562,149.11827
1,AINSLIE,0,15,21,1,0,1,2,34,1,8,14,218,78,3,2,0,136,34,-35.26222,149.14655
2,AMAROO,0,9,19,5,0,1,0,26,0,3,7,49,28,4,2,3,83,17,-35.16922,149.12637
3,ARANDA,0,8,3,1,0,2,1,12,0,3,3,103,13,199,14,3,30,127,-35.25804,149.08293
4,BANKS,0,5,2,3,0,1,0,16,1,0,8,59,33,1,0,0,35,4,-35.47004,149.09771


In [35]:
# create map of Canberra using latitude and longitude values
map_cbr_crime = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(cbr_crime['Latitude'], cbr_crime['Longitude'], cbr_crime['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(map_cbr_crime)  
    
map_cbr_crime

In [40]:
map_hooray = folium.Map(location=[latitude, longitude], zoom_start=10)

# Ensure you're handing it floats
cbr_crime['Latitude'] = cbr_crime['Latitude'].astype(float)
cbr_crime['Longitude'] = cbr_crime['Longitude'].astype(float)

heat_df = cbr_crime[['Latitude', 'Longitude']]

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in cbr_crime.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray

In [38]:
heat_data

[[-35.28561999999994, 149.11827000000005],
 [-35.26221999999996, 149.14655000000005],
 [-35.16921999999994, 149.12637000000007],
 [-35.25803999999994, 149.08293000000003],
 [-35.47003999999998, 149.09771000000012],
 [-35.30828999999994, 149.13354000000004],
 [-35.343219999999974, 149.21217000000001],
 [-35.23944999999998, 149.0634500000001],
 [-35.27360999999996, 149.0975400000001],
 [-35.15749999999997, 149.14303000000007],
 [-35.43090999999998, 149.0802000000001],
 [-35.27461999999997, 149.1383300000001],
 [-35.24571999999995, 149.0926800000001],
 [-35.43791999999996, 149.1092000000001],
 [-35.29472999999996, 149.16005000000007],
 [-35.306539999999984, 149.12655000000007],
 [-35.16863999999998, 149.09315000000004],
 [-35.35617999999994, 149.0407600000001],
 [-35.19797999999997, 149.03235000000006],
 [-35.352499999999964, 149.0754700000001],
 [-35.41901999999993, 149.1237400000001],
 [-35.28006999999997, 149.13093000000003],
 [-35.45775999999995, 149.09733000000006],
 [-35.26024999999